# Research Organization Registry API in Python

by Michael T. Moen

The Research Organization Registry (ROR) API provides persistent identifiers for research organizations.

- Documentation
    - <a href="https://ror.readme.io/docs/rest-api" target="_blank">ROR API Documentation</a>
- Data Reuse
    - <a href="https://ror.readme.io/docs/ror-basics#what-is-ror" target="_blank">ROR API License Info</a>
    - The ROR data is licensed under the Creative Commons' <a href="https://creativecommons.org/publicdomain/zero/1.0/" target="_blank">CC0 license</a>.

*These recipe examples were tested on March 28, 2025.*

**_NOTE:_** The ROR API limits requests to a maximum of 2000 requests in a 5-minute period.

## Setup

### Import Libraries

The following external libraries need to be installed into your environment to run the code examples in this tutorial:

- <a href="https://github.com/ipython/ipykernel" target="_blank">ipykernel</a>
- <a href="https://github.com/psf/requests" target="_blank">requests</a>

We import the libraries used in this tutorial below:

In [1]:
import requests
from time import sleep

### ROR Data Dump

When working with larger datasets, consider using the <a href="https://ror.readme.io/docs/data-dump" target="_blank">ROR data dump</a>.

## 1. Searching with Queries

This first example uses the `query` parameter of the ROR API to search for an institution by name. In this example, we'll search for The University of Alabama:

In [2]:
# The search query is the institution name
institution = 'University of Alabama'

# Construct URL and send HTTP request
url = f'https://api.ror.org/organizations?query={institution}'
response = requests.get(url).json()

# Print total number of results and number of results in page
print(f'Total number of results: {response["number_of_results"]}')
print(f'Page length: {len(response["items"])}')

Total number of results: 27751
Page length: 20


The results indicate that the query produced thousands of results, but only the data for 20 institutions were returned in this query. However, the top result is exactly what we were looking for:

In [3]:
# Display data of the top search result
response['items'][0]

{'id': 'https://ror.org/03xrrjk67',
 'name': 'University of Alabama',
 'email_address': None,
 'ip_addresses': [],
 'established': 1831,
 'types': ['Education', 'Funder'],
 'relationships': [{'label': 'University of Alabama System',
   'type': 'Parent',
   'id': 'https://ror.org/051fvmk98'},
  {'label': 'Mississippi Alabama Sea Grant Consortium',
   'type': 'Related',
   'id': 'https://ror.org/04vzsq290'}],
 'addresses': [{'lat': 33.20984,
   'lng': -87.56917,
   'state': None,
   'state_code': None,
   'city': 'Tuscaloosa',
   'geonames_city': {'id': 4094455,
    'city': 'Tuscaloosa',
    'geonames_admin1': {'name': 'Alabama',
     'id': None,
     'ascii_name': None,
     'code': 'US.AL'},
    'geonames_admin2': {'name': None,
     'id': None,
     'ascii_name': None,
     'code': None},
    'license': {'attribution': 'Data from geonames.org under a CC-BY 3.0 license',
     'license': 'http://creativecommons.org/licenses/by/3.0/'},
    'nuts_level1': {'name': None, 'code': None},
   

The following code produces the name, ROR ID, city, and wikipedia URL of the top result of the query:

In [4]:
response['items'][0]['name']

'University of Alabama'

In [5]:
response['items'][0]['id']

'https://ror.org/03xrrjk67'

In [6]:
response['items'][0]['addresses'][0]['city']

'Tuscaloosa'

In [7]:
response['items'][0]['wikipedia_url']

'http://en.wikipedia.org/wiki/University_of_Alabama'

### Searching by Alternate Names

The example below uses abbreviated forms of the full names of universities when searching:

In [8]:
# List of institutions to be searched
institutions = [
    'University of Alabama Tuscaloosa',
    'Missouri',
    'Dartmouth',
    'Oxford',
    'UCLA'
]

# Send an HTTP request for each institution
for institution in institutions:
    url = f'https://api.ror.org/organizations?query={institution}'
    search_data = requests.get(url).json()

    # Print the search term and the name of its top result
    print(f'{institution}: {search_data["items"][0]["name"]}')

    # Stagger requests to be nicer on the ROR servers
    sleep(0.5)

University of Alabama Tuscaloosa: University of Alabama
Missouri: Missouri Southern State University
Dartmouth: Dartmouth Hospital
Oxford: Oxford Optronix
UCLA: Universidad Centroccidental Lisandro Alvarado


The top results of the queries above are probably not what you would have expected. The example below remedies these issues by having more clearly defined search strings:

In [9]:
# List of institutions to be searched
institutions = [
    'University of Alabama Tuscaloosa',
    'University of Missouri',
    'Dartmouth College',
    'University of Oxford',
    'University of California Los Angeles'
]

# Send an HTTP request for each institution
for institution in institutions:
    url = f'https://api.ror.org/organizations?query={institution}'
    search_data = requests.get(url).json()

    # Print the search term and the name of its top result
    print(f'{institution}: {search_data["items"][0]["name"]}')

    # Stagger requests to be nicer on the ROR servers
    sleep(0.5)

University of Alabama Tuscaloosa: University of Alabama
University of Missouri: University of Missouri
Dartmouth College: Dartmouth College
University of Oxford: University of Oxford
University of California Los Angeles: University of California, Los Angeles


## 2. Searching with Filters

The ROR API also allows searches to be performed with the `filter` parameter, which can take 3 arguments: `status`, `types`, and `country`. For more information on what values these arguments can take, read the <a href="https://ror.org/tutorials/intro-ror-api/#filtering-results" target="_blank">ROR documentation</a>.

In [10]:
filters = ','.join([
    'country.country_name:United States',
    'types:Education',
    'status:Active'
])

# URL constructed with the filters
url = f'https://api.ror.org/organizations?filter={filters}'
response = requests.get(url).json()

# Display number of results
response['number_of_results']

4331

### Paging through a Result

The example below pages through the results to find the names and ROR IDs of the first 100 institutions returned using the filter:

In [11]:
# Filters to are separated by commas
filters = ','.join([
    'country.country_name:United States',
    'types:Education',
    'status:Active'
])

# URL constructed with the filters
url = f'https://api.ror.org/organizations?filter={filters}'
response = requests.get(url).json()

# Calculate number of pages in result
total_pages = (response['number_of_results'] // len(response['items'])) + 1

# Store resulting names in a dictionary
institution_rors = {}

# Limited to first 5 pages for this tutorial
for page_number in range(total_pages)[:5]:
    url = f'https://api.ror.org/organizations?filter={filters}&page={page_number+1}'
    search_data = requests.get(url).json()

    # Add institution names and ROR IDs to the institution_results list
    for result in search_data['items']:
        institution_rors[result['name']] = result['id']

    # Stagger requests to be nicer on the ROR servers
    sleep(0.5)

# Display first 100 results
for name, ror_id in sorted(institution_rors.items()):
    print(f'{name}: {ror_id}')

Albert Einstein Academy: https://ror.org/01fhhqb25
Albuquerque Academy: https://ror.org/04bty2w72
American College of Veterinary Ophthalmologists: https://ror.org/010m1h651
American University of Beirut New York Office: https://ror.org/05t3agv77
Arizona School for the Arts: https://ror.org/00a0wam85
Arkansas State University System: https://ror.org/0152qkr29
Asbestos Institute: https://ror.org/058kkmh35
Avraham Y. Goldratt Institute: https://ror.org/00me50c36
BMCC Tribeca Performing Arts Center: https://ror.org/021rrz441
Beck Institute for Cognitive Behavior Therapy: https://ror.org/0271pep11
Beth Mardutho: https://ror.org/01agawy76
Brown-Spath & Associates: https://ror.org/05h7bqz94
Bucks County Intermediate Unit: https://ror.org/052bwdm80
Cacapon Institute: https://ror.org/02bt8sv61
California Northstate University: https://ror.org/03h0d2228
Chattahoochee Valley Community College: https://ror.org/052rvpm80
Chhandam School of Kathak: https://ror.org/01z2dq675
Chicago Arts Partnerships

The resulting dictionary can be used to find the ROR of an institution based on its name:

In [12]:
institution_rors['Purdue University Fort Wayne']

'https://ror.org/04c4hz115'

## 3. Searching with Queries and Filters

The `filter` and `query` parameters can both be used in a single request. In this example, we filter the results of the query "Birmingham" to only include institutions from the United States:

In [13]:
# Filter results to the United States
filter = 'country.country_name:United States'

# Search term
query = 'Birmingham'

# URL constructed with the filters
url = f'https://api.ror.org/organizations?query={query}&filter={filter}'
response = requests.get(url).json()

# Display number of results
response['number_of_results']

12

In [14]:
# Filter results to the United States
filter = f'country.country_name:United States'

# Search term
query = 'Birmingham'

# URL constructed with the filters
url = f'https://api.ror.org/organizations?query={query}&filter={filter}'
response = requests.get(url).json()

# Calculate number of pages in result
total_pages = (response['number_of_results'] // len(response['items'])) + 1

# Store resulting names in a dictionary
institution_rors = {}

# Limited to first 5 pages for this tutorial
for page_no in range(total_pages):
    url = f'https://api.ror.org/organizations?query={query}&filter={filter}&page={page_no+1}'
    search_data = requests.get(url).json()

    # Add institution names and ROR IDs to the institution_results list
    for result in search_data['items']:
        institution_rors[result['name']] = result['id']

    # Stagger requests to be nicer on the ROR servers
    sleep(0.5)

# Display first 100 results
for name, ror_id in sorted(institution_rors.items()):
    print(f'{name}: {ror_id}')

Alabama Audubon: https://ror.org/02qbyex13
Birmingham Bloomfield Community Coalition: https://ror.org/004mx7t23
Birmingham Civil Rights Institute: https://ror.org/00fqce595
Birmingham Museum of Art: https://ror.org/030y6zg68
Birmingham Public Library: https://ror.org/05czff141
Birmingham VA Medical Center: https://ror.org/0242qs713
Birmingham–Southern College: https://ror.org/006g42111
St. Vincent's Birmingham: https://ror.org/000crk757
UAB Medicine: https://ror.org/036554539
University of Alabama at Birmingham: https://ror.org/008s83205
University of Alabama at Birmingham Hospital: https://ror.org/01rm42p40
Vision Specialists of Michigan: https://ror.org/02awhp844
